In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn

In [0]:
data = pd.read_csv('/gdrive/My Drive/iris.data')

In [0]:
data['x0']=1
data = data[['x0','x1','x2','x3','x4','y']]
y = data['y']
data_y = np.array(y)

In [5]:
label_encoder = preprocessing.LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_y)
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
y = onehot_encoded

In [0]:
x = data[['x0','x1','x2','x3','x4']]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = x.values

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [0]:
import random

In [0]:
def activation(x):
    return 1/(1+np.exp(-x))

In [0]:
def feedforward_2(x,w1,w2):#w1 = 3x5 , w2 = 3 x 4 
    u1 = np.zeros((4,1))
    u2 = np.zeros((3,1))
    x = x.reshape((5,1))
    u1[1:] = np.dot(w1, x)
    u1[0] = 1
    v1 = activation(u1)
    v1[0] = 1
    u2 = np.dot(w2,v1)
    v2 = activation(u2)
    return v1,v2

In [0]:
def backprop_2(x,t,v1,v2,w1,w2):
    x = x.reshape((5,1))
    t = t.reshape((3,1))
    v1 = v1.reshape((4,1))
    v2 = v2.reshape((3,1))
    
    delta_2 = (t-v2)*(v2)*(1.0-v2)
    delta_1 = np.zeros((3,1))
    
    grad_w2 = np.zeros((3,4))
    grad_w1 = np.zeros((3,5))
    
    grad_w2 = lr * np.dot(delta_2,v1.T)
    
    delta_1 = np.dot(w2.T,delta_2)
    delta_1 = delta_1 * v1 * (1.0-v1)
    delta_1 = delta_1[1:]
    
    grad_w1 = lr * np.dot(delta_1,x.T)
    
    w1 = w1 + grad_w1
    w2 = w2 + grad_w2
    
    return w1,w2

In [0]:
def ann_2(w1,w2):
    #w1 = np.random.random((3,5))
    #w2 = np.random.random((3,4))
    for _ in range(3000):
        for i in range(X_train.shape[0]):
            v1,v2 = feedforward_2(X_train[i],w1,w2)
            w1,w2 = backprop_2(X_train[i],y_train[i],v1,v2,w1,w2)
    return w1,w2

In [0]:
random.seed(30)

In [0]:
w1 = np.random.random((3,5))
w2 = np.random.random((3,4))

In [0]:
w1 = np.array([[0.59648215,0.86694168,0.3121895,0.13625315,0.94715007],
 [0.20637245,0.89499001,0.59337413,0.35704101,0.5970807],
 [0.63183141,0.22993348,0.29821053,0.1594374,0.6198028]])

w2 = np.array([[0.7405462,0.32821044 ,0.1879626,0.53301198],
 [0.04783333, 0.54862865,0.11447806,0.49499292],
 [0.47451542, 0.71711121 ,0.30571364, 0.15988641]])

In [0]:
lr=0.1
w1_n,w2_n = ann_2(w1,w2)

In [0]:
w1_n

array([[ 5.11221845e-05, -7.09072453e-01, -2.65860343e+00,
         3.60331278e+00,  2.45995930e+00],
       [ 2.81840989e-01,  1.25439002e+00,  8.41860326e-01,
         4.48240628e-01,  6.06641674e-01],
       [-8.85296736e+00, -5.47686176e+00, -1.38498061e+01,
         1.26444880e+01,  1.35300797e+01]])

In [0]:
w2_n

array([[  2.84588875, -10.24993182,   2.26078024, -25.14455597],
       [ -2.56579623,  11.40168459,  -2.38714894,  -9.54310857],
       [ -6.5981536 ,  -6.28042899,  -6.7519145 ,  30.23044198]])

In [0]:
pred = []
target=[]
error=[]
for i in range (X_test.shape[0]):
    v1,v2 = feedforward_2(X_test[i],w1_n,w2_n)
    p = np.argmax(v2)
    pred.append(p)
    t = np.argmax(y_test[i])
    target.append(t)
    if(p!=t):
        error.append(1)
    else:
        error.append(0)

In [0]:
err = np.array(error)

In [21]:
print ("accuracy = {}%".format(100 - (np.sum(err)/X_test.shape[0])*100))

accuracy = 97.36842105263158%


In [42]:
n = int(input("Enter number of hidden layers"))
units=[5]
for i in range(n):
    t = int(input("Enter the number of units in layer {} ".format(i+1) ))
    units.append(t+1)
units.append(3)
units

Enter number of hidden layers2
Enter the number of units in layer 1 3
Enter the number of units in layer 2 3


[5, 4, 4, 3]

In [0]:
def feedforward(x , n , units,weights):
    op=[]
    op.append(x)
    ip =x
    for i in range(0,n-1):
        if(i!=n-2):
            u = np.zeros((weights[i].shape[0] + 1))
            u[1:] = np.dot(weights[i],ip)
            u[0] = 1

        else:
            u = np.zeros((weights[i].shape[0]))
            u = np.dot(weights[i],ip)
        v = activation(u)
        v[0] = 1
        op.append(v)
        ip = v
    return op

In [0]:
def backprop(x,v,t,weights,n,units):
    x = x.reshape((units[0],1))
    t = t.reshape((3,1))
    deltas=[]
    grads=[]
    modified=[]
    for i in range(n-1,-1,-1):
        op = v[i]
        op = op.reshape((units[i],1))
        n_units = units[i]
        if (i== n-1):            
            delta = (t-op)*(op)*(1.0-op)
            deltas.append(delta)
        else:
            w = weights[i]
            delta_next_layer = deltas[-1]
            delta = np.dot(w.T,delta_next_layer)
            delta = delta * op * (1-op)
            delta = delta[1:]            
            grad = lr * np.dot(deltas[-1],op.T)
            grads.append(grad)
    
    grads = grads[::-1]
    for (x,y) in zip(weights,grads):
        new = x - lr*y
        modified.append(new)
    return modified

In [0]:
weights=[]
for i in range(1,len(units)):
    if(i!=len(units)-1):
        weight = np.random.random((units[i]-1,units[i-1]))
    else:
        weight = np.random.random((units[i],units[i-1]))
    weights.append(weight)

In [0]:
def ann(weights):
    for _ in range(3000):
        for i in range(X_train.shape[0]):
            v = feedforward(X_train[i],4,units,weights)
            weights = backprop(X_train[i],v,y_train[i],weights,4,units)
    return weights

In [0]:
new_weights = ann(weights)

In [54]:
new_weights

[array([[ 0.47164826,  0.93593144,  0.88242051,  0.5760448 ,  0.80219867],
        [ 2.86039518, 11.60876706,  6.98336928,  7.24397508,  2.67999559],
        [ 2.6595236 , 10.465394  ,  6.33835724,  5.98512413,  2.0552335 ]]),
 array([[0.20455566, 0.71556944, 0.07415207, 0.65511248],
        [2.0554511 , 2.47883163, 2.1680343 , 2.86973234],
        [2.6744332 , 2.14886562, 2.90587088, 2.14676243]]),
 array([[0.14573964, 0.95621429, 0.13693573, 0.73455618],
        [2.25664691, 2.1409397 , 2.67056111, 2.27675847],
        [2.21592831, 2.45859688, 2.12538919, 2.60992955]])]

In [0]:
pred = []
target=[]
error=[]
for i in range (X_test.shape[0]):
    v1,v2 = feedforward_2(X_test[i],w1_n,w2_n)
    p = np.argmax(v2)
    pred.append(p)
    t = np.argmax(y_test[i])
    target.append(t)
    if(p!=t):
        error.append(1)
    else:
        error.append(0)

In [56]:
err = np.array(error)
print ("accuracy = {}%".format(100 - (np.sum(err)/X_test.shape[0])*100))

accuracy = 97.36842105263158%
